# Predictions

Instructions:

i) Paste the test images folder in the same folder consisting this code file.

ii) Rename the folder to 'test'.

Run the codes sequentially

In [ ]:

import os
import numpy as np
import torch

import torch.nn as nn
import torch.nn.functional as F

import torch.utils.data as data
import torchvision
from torchvision import transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import cv2 


In [ ]:
# Convert the output labels to pixel-wise classes
size=400
def c2g(cn):
    cn1 = np.reshape(cn, (1, 1, 3));
    cn = cv2.cvtColor(cn1, cv2.COLOR_BGR2GRAY);
    del cn1
    return cn;

colors = [];
colors.append(c2g(np.array([0, 0, 0], dtype = 'uint8')));
colors.append(c2g(np.array([255, 255, 255], dtype = 'uint8')));

# colors.append(c2g(np.array([0, 0, 0], dtype = 'uint8')));

import torch
from PIL import Image
from torch import Tensor
from torchvision.transforms import transforms
import torchvision.transforms.functional as F

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import numbers
from abc import abstractmethod
from typing import Tuple
import random

# print(c0)
ClassesColors = {
    (0,0,0): 0 ,
     (255,255,255): 1
            
    }

potsdam_map = {v: k for k,v in ClassesColors.items()}

# 

def class_pixel(label_img):
    aa,bb = label_img.shape

    
    class_pix = np.ones([aa, bb, 1], dtype = int);
    for index, c in enumerate(colors):
        
        class_pix[label_img == c] = index; 

    

    return class_pix
def label_img_list(img_list):
    images = [];
    for image in img_list:
        images.append(class_pixel(image));
    del img_list
    return images;


In [ ]:

import torch
from torch import nn
from torch.nn import functional as F

from numpy.linalg import svd
from numpy.random import normal
from math import sqrt
import torchvision.models as models

import torch

import torch.nn as nn
import torch.nn.functional as F
from network.utils import IntermediateLayerGetter
from network._deeplab import DeepLabHead, DeepLabHeadV3Plus, DeepLabV3
from network.backbone import resnet
from network.backbone import mobilenetv2

def _segm_resnet(name, backbone_name, num_classes, output_stride, pretrained_backbone):

    if output_stride==8:
        replace_stride_with_dilation=[False, True, True]
        aspp_dilate = [12, 24, 36]
    else:
        replace_stride_with_dilation=[False, False, True]
        aspp_dilate = [6, 12, 18]

    backbone = resnet.__dict__[backbone_name](
        pretrained=pretrained_backbone,
        replace_stride_with_dilation=replace_stride_with_dilation)
    
    inplanes = 2048
    low_level_planes = 256

    if name=='deeplabv3plus':
        return_layers = {'layer4': 'out', 'layer1': 'low_level'}
        classifier = DeepLabHeadV3Plus(inplanes, low_level_planes, num_classes, aspp_dilate)
    elif name=='deeplabv3':
        return_layers = {'layer4': 'out'}
        classifier = DeepLabHead(inplanes , num_classes, aspp_dilate)
    backbone = IntermediateLayerGetter(backbone, return_layers=return_layers)

    model = DeepLabV3(backbone, classifier)
    return model

def _segm_mobilenet(name, backbone_name, num_classes, output_stride, pretrained_backbone):
    if output_stride==8:
        aspp_dilate = [12, 24, 36]
    else:
        aspp_dilate = [6, 12, 18]

    backbone = mobilenetv2.mobilenet_v2(pretrained=pretrained_backbone, output_stride=output_stride)
    
    # rename layers
    backbone.low_level_features = backbone.features[0:4]
    backbone.high_level_features = backbone.features[4:-1]
    backbone.features = None
    backbone.classifier = None

    inplanes = 320
    low_level_planes = 24
    
    if name=='deeplabv3plus':
        return_layers = {'high_level_features': 'out', 'low_level_features': 'low_level'}
        classifier = DeepLabHeadV3Plus(inplanes, low_level_planes, num_classes, aspp_dilate)
    elif name=='deeplabv3':
        return_layers = {'high_level_features': 'out'}
        classifier = DeepLabHead(inplanes , num_classes, aspp_dilate)
    backbone = IntermediateLayerGetter(backbone, return_layers=return_layers)

    model = DeepLabV3(backbone, classifier)
    return model

def _load_model(arch_type, backbone, num_classes, output_stride, pretrained_backbone):

    if backbone=='mobilenetv2':
        model = _segm_mobilenet(arch_type, backbone, num_classes, output_stride=output_stride, pretrained_backbone=pretrained_backbone)
    elif backbone.startswith('resnet'):
        model = _segm_resnet(arch_type, backbone, num_classes, output_stride=output_stride, pretrained_backbone=pretrained_backbone)
    else:
        model = _segm_resnet(arch_type, backbone, num_classes, output_stride=output_stride, pretrained_backbone=pretrained_backbone)
    return model


# Deeplab v3

def deeplabv3_resnet50(num_classes=21, output_stride=8, pretrained_backbone=True):
    """Constructs a DeepLabV3 model with a ResNet-50 backbone.
    Args:
        num_classes (int): number of classes.
        output_stride (int): output stride for deeplab.
        pretrained_backbone (bool): If True, use the pretrained backbone.
    """
    return _load_model('deeplabv3', 'resnet50', num_classes, output_stride=output_stride, pretrained_backbone=pretrained_backbone)

def deeplabv3_resnet101(num_classes=21, output_stride=8, pretrained_backbone=True):
    """Constructs a DeepLabV3 model with a ResNet-101 backbone.
    Args:
        num_classes (int): number of classes.
        output_stride (int): output stride for deeplab.
        pretrained_backbone (bool): If True, use the pretrained backbone.
    """
    return _load_model('deeplabv3', 'resnet101', num_classes, output_stride=output_stride, pretrained_backbone=pretrained_backbone)

def deeplabv3_mobilenet(num_classes=21, output_stride=8, pretrained_backbone=True, **kwargs):
    """Constructs a DeepLabV3 model with a MobileNetv2 backbone.
    Args:
        num_classes (int): number of classes.
        output_stride (int): output stride for deeplab.
        pretrained_backbone (bool): If True, use the pretrained backbone.
    """
    return _load_model('deeplabv3', 'mobilenetv2', num_classes, output_stride=output_stride, pretrained_backbone=pretrained_backbone)


# Deeplab v3+

def deeplabv3plus_resnet50(num_classes=23, output_stride=2, pretrained_backbone=True):
    """Constructs a DeepLabV3 model with a ResNet-50 backbone.
    Args:
        num_classes (int): number of classes.
        output_stride (int): output stride for deeplab.
        pretrained_backbone (bool): If True, use the pretrained backbone.
    """
    return _load_model('deeplabv3plus', 'resnet50', num_classes, output_stride=output_stride, pretrained_backbone=pretrained_backbone)

def deeplabv3plus_resnext50_32x4d(num_classes=2, output_stride=2, pretrained_backbone=True):
    """Constructs a DeepLabV3 model with a ResNet-50 backbone.
    Args:
        num_classes (int): number of classes.
        output_stride (int): output stride for deeplab.
        pretrained_backbone (bool): If True, use the pretrained backbone.
    """
    return _load_model('deeplabv3plus', 'resnext50_32x4d', num_classes, output_stride=output_stride, pretrained_backbone=pretrained_backbone)
def deeplabv3plus_resnext101_32x8d(num_classes=2, output_stride=2, pretrained_backbone=True):
    """Constructs a DeepLabV3 model with a ResNet-50 backbone.
    Args:
        num_classes (int): number of classes.
        output_stride (int): output stride for deeplab.
        pretrained_backbone (bool): If True, use the pretrained backbone.
    """
    return _load_model('deeplabv3plus', 'resnext101_32x8d', num_classes, output_stride=output_stride, pretrained_backbone=pretrained_backbone)
def deeplabv3plus_resnet101(num_classes=21, output_stride=8, pretrained_backbone=True):
    """Constructs a DeepLabV3+ model with a ResNet-101 backbone.
    Args:
        num_classes (int): number of classes.
        output_stride (int): output stride for deeplab.
        pretrained_backbone (bool): If True, use the pretrained backbone.
    """
    return _load_model('deeplabv3plus', 'resnet101', num_classes, output_stride=output_stride, pretrained_backbone=pretrained_backbone)


def deeplabv3plus_mobilenet(num_classes=21, output_stride=8, pretrained_backbone=True):
    """Constructs a DeepLabV3+ model with a MobileNetv2 backbone.
    Args:
        num_classes (int): number of classes.
        output_stride (int): output stride for deeplab.
        pretrained_backbone (bool): If True, use the pretrained backbone.
    """
    return _load_model('deeplabv3plus', 'mobilenetv2', num_classes, output_stride=output_stride, pretrained_backbone=pretrained_backbone)
import os

# if __name__ == '__main__':
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#     ntimes = 10
#     import time
#     import os
#     model =  deeplabv3plus_resnet101(num_classes=2, output_stride=2, pretrained_backbone=True)
#     model.to(device)

#     with torch.no_grad():
#         x = torch.randn(2,3,400,400)
#         x = x.to(device)
#         start = time.time()
#         for i in range(ntimes):
#             model(x)
#         print('fps is :', 1.0/((time.time() - start)/ntimes))

# Load Weights

If testing for kvasir instrument change chal1.pth to chal2.pth

If testing for kvasir seg then its set default to chal1.pth

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu");
print(device)
UNET = deeplabv3plus_resnet101(num_classes=2, output_stride=2, pretrained_backbone=True)
UNET.to(device);

# weightsInit(UNET)
PATH = 'weights/chal1.pth';
UNET.load_state_dict(torch.load(PATH))
# !zip -r /content/file.zip /content/test_input_data

In [ ]:

colors = [];
colors.append(np.array([0, 0, 0], dtype = 'uint8'));
colors.append(np.array([255, 255, 255], dtype = 'uint8'));

ClassesColors = {
    (0,0,0): 0 ,
     (255,255,255): 1
            
    }



def test(op_img):
    class_pix = np.ones([620, 2026, 3], dtype = 'uint8');
    for index, c in enumerate(colors):
        class_pix[op_img == index] = c; # Vectorized masking is much much faster
    return class_pix.reshape((620, 2026, 3))
import numpy as np

color_mapping = {
     0 : [0,0,0], 
     1 : [255,255,255]
}

    
semantic_map = {
     0 : 'asphalt', 
     1 : 'gravel'
}

def visualize_prediction(prediction):
    color_image = np.zeros((prediction.shape[0], prediction.shape[1], 3))
    for color_id in color_mapping.keys():
        color_image[prediction == color_id] = color_mapping[color_id]
    return color_image.astype(np.uint8)


def get_class_names(class_ids):
    return [semantic_map[i] for i in class_ids]
def corr(x): # To get proper correspondence between the outputs and the labels
    x = x.cpu();
    x = x.detach().numpy(); # Detach() was used as one can't convert a pytorch tensor to a numpy array if
    # required_grad is set True for that variable
    x = x.argmax(axis = 1);
    return x;
def corrr(x): # To get proper correspondence between the outputs and the labels
    x = x.cpu();
    x = x.detach().numpy(); 
    return x;

In [ ]:
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'
colors = [];
colors.append(np.array([0, 0, 0], dtype = 'uint8'));
colors.append(np.array([255, 255, 255], dtype = 'uint8'));
colorss=[];
colorss.append(np.array([0, 100, 0], dtype = 'uint8'));
colorss.append(np.array([0, 0, 255], dtype = 'uint8'));


ClassesColors = {
    (0,0,0): 0 ,
     (255,255,255): 1
            
    }


sen = [];

def testss(img1,img2):
    for i in range(400):
      for j in range(400):
        if img1[i][j]==1:
          sen.append(np.array(img2[i][j], dtype = 'uint8'));
    print('pass')
    

def test(op_img):
    class_pix = np.ones([620, 2026, 3], dtype = 'uint8');
    for index, c in enumerate(colors):
        class_pix[op_img == index] = c; # Vectorized masking is much much faster
    return class_pix.reshape((620, 2026, 3))
import numpy as np

color_mapping = {
     0 : [0,0,0], 
     1 : [255,255,255]
}
colors_mapping = {
     0 : [0,191,255], 
     1 : [255,0,0]
}
    
semantic_map = {
     0 : 'asphalt', 
     1 : 'gravel'}

def visualize_prediction(prediction):
    color_image = np.zeros((prediction.shape[0], prediction.shape[1], 3))
    for color_id in color_mapping.keys():
        color_image[prediction == color_id] = color_mapping[color_id]
    return color_image.astype(np.uint8)
def visualize_predictions(prediction):
    color_image = np.zeros((prediction.shape[0], prediction.shape[1], 3))
    for color_id in colors_mapping.keys():
        color_image[prediction == color_id] = colors_mapping[color_id]
    return color_image.astype(np.uint8)
def visualize_predictionss(prediction):
    color_image = np.zeros((prediction.shape[0], prediction.shape[1], 3))
    for color_id in colors_mapping.keys():
        color_image[prediction == color_id] = colors_mapping[color_id]
    
    return color_image.astype(np.uint8)

def get_class_names(class_ids):
    return [semantic_map[i] for i in class_ids]
def corr(x): # To get proper correspondence between the outputs and the labels
    x = x.cpu();
    x = x.detach().numpy(); # Detach() was used as one can't convert a pytorch tensor to a numpy array if
    # required_grad is set True for that variable
    x = x.argmax(axis = 1);
    return x;
def class_pixel(label_img):
    # print(label_img.shape)
    
    aa,bb = label_img.shape
    
    class_pix = np.ones([aa, bb, 1], dtype = int);
    for index, c in enumerate(colors):
      # class_pix[label_img ==c] = index; 
        
        if index==0:
          # print('index',index)
          # print('color',c)

          class_pix[label_img <=122] = index; 
        else:
          class_pix[label_img >=123] = index; 

        

    # print(class_pix)
    

    return class_pix




In [ ]:
Predictions

In [ ]:
# """""""" PREDICTIONS """"""""""""""""""""""""""
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'
def matplotlib_imshow(display_list):
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        # plt.title(title[i])
#         print(i)
        plt.imshow(display_list[i])
        plt.axis('off')
    plt.show()

import matplotlib.pyplot as plt

# import numpy
CUDA_LAUNCH_BLOCKING=1
los=0
kb=0
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
transform_img = transforms.Compose([ 
    transforms.ToTensor()]);
# from google.colab.patches import cv2_imshow
kb=los

with torch.no_grad():
  # model.eval()
  c = 0;
  folder = 'MedAI_2021_Polyp_Segmentation_Test_Dataset/'
  images = [];
  filenam =[];
  submission={}
  UNET.eval()
  transform_img = transforms.Compose([ 
      transforms.ToTensor()]);
  res=0
  for name in os.listdir(folder): # List all the filenames in the folde
      print('nwname ',name)
      c = c + 1;
      print(c);
      img2 = cv2.imread(os.path.join(folder, name)); # Join the link of the folder and filename
      # print(img.shape)
      a1,d1,c1=img2.shape
      
      inputs1 = cv2.resize(img2, (400, 400))
      img3=cv2.cvtColor(img2, cv2.COLOR_BGR2RGB);
      
      inputs=transform_img(inputs1)
      inputs=inputs.unsqueeze(0).to(device)

      outputs = UNET.forward(inputs);
      
      inputs=inputs.squeeze(0)
      inputs= np.array(inputs.cpu())
      inputs = inputs.reshape(size,size,3)
      outputs = corr(outputs);
      
      outputs = outputs.reshape(1, size, size);
      
      color_pre=outputs
      
      dd,aa,bb = color_pre.shape
      # d1,a1,b1 =labels[0].shape

      color_pre=np.array(color_pre).reshape(aa,bb)


      color_pre=visualize_prediction(color_pre)
      img = cv2.resize(color_pre, (d1,a1))
      
      matplotlib_imshow([img3,img])
      # print(img.shape)
    
      
      cv2.imwrite('results/'+name,img)  #       CREATE THE FOLDER AND AND SAVE THE PREDICTED IMAGES IN THE FOLDER
    
      
      